In [ ]:
import menpo3d.io as m3io
import menpo.io as mio
from menpo.shape import PointCloud, ColouredTriMesh
from menpo.landmark import face_ibug_68_to_face_ibug_68
import numpy as np

<h3> Load the data (Mesh, landmarks and model) </h3>

In [ ]:
mesh = m3io.import_mesh('../menpo3d/data/james.obj')
lms = m3io.import_landmark_file('../menpo3d/data/james.ljson')['LJSON']

# Load model and its landmarks indices 
model =  mio.import_pickle('link_to_model')['model']
lms_indices = [21868, 22404, 22298, 22327, 43430, 45175, 46312, 47132, 47911, 48692,
               49737, 51376, 53136, 32516, 32616, 32205, 32701, 38910, 39396, 39693,
               39934, 40131, 40843, 41006, 41179, 41430, 13399, 8161, 8172, 8179, 8185,
               5622, 6881, 8202, 9403, 10764, 1831, 3887, 5049, 6214, 4805, 3643, 9955,
               11095, 12255, 14197, 12397, 11366, 5779, 6024, 7014, 8215, 9294, 10267,
               10922, 9556, 8836, 8236, 7636, 6794, 5905, 7264, 8223, 9063, 10404, 8828,
               8228, 7509]

<h3> Create new random instances </h3>

In [ ]:
cov = np.diag(model.eigenvalues)
model_mean = model.mean()
synthetic_weights = np.random.multivariate_normal(np.zeros(model.n_active_components),
                                                 cov, 1000)
random_mesh = model.instance(synthetic_weights[5])

<h3> Show the mesh </h3>

In [ ]:
# Default values for TriMesh, TextureMesh viewer are
# figure_id None
# new_figure True
# in that case an automatic figure_id will be given
# with 'Figure_{n}' format
# n will be an increased integer starting from zero
mesh.view() # wait a bit before magic happens

<h3> Show the mesh and landmarks </h3>

In [ ]:
mesh.view(figure_id='James')

In [ ]:
# Add landmarks to figure with id James
lms_poincloud = PointCloud(lms.points)
lms_poincloud.view(figure_id='James',new_figure=False)

In [ ]:
# Add landmarks to figure with id Figure_0
lms_poincloud.view(figure_id='Figure_0', new_figure=False)

<h3> Show a mesh that has landmarks </h3>

In [ ]:
mesh.landmarks = lms

In [ ]:
pc = PointCloud(lms.points)

In [ ]:
pc.view(new_figure=True, alpha=0.5, render_numbering=True, numbers_colour='g')

In [ ]:
lms.view(new_figure=True, render_numbering=True, numbers_colour='g' )

In [ ]:
# The mesh has now landmarks, so they would be plotted as well
# the figure id is now Figure_2
mesh.view()

In [ ]:
# Show the TexturedMesh without texture
mesh.view(render_texture=False)

In [ ]:
mesh.view(render_texture=False, mesh_type='wireframe')

In [ ]:
random_mesh.view(mesh_type='surface')

<h3> HeatMaps </h3>

In [ ]:
# Heatmap between a random mesh and mean mesh
random_mesh.heatmap(model_mean)

In [ ]:
# Heatmap with statistics 
# Be careful, since we have already drawn a heatmap between
# random and mean, we should use another name for figure
random_mesh.heatmap(model_mean, show_statistics=True, figure_id='Heatmap2')

In [ ]:
# Heatmap with landmarks
model_mean.landmarks = face_ibug_68_to_face_ibug_68(PointCloud(model_mean.points[lms_indices]))
model_mean.heatmap(random_mesh, inline=True, show_statistics=True)

In [ ]:
random_mesh_landmarks = face_ibug_68_to_face_ibug_68(random_mesh.points[lms_indices])
random_mesh_landmarks.view(inline=True, new_figure=False, figure_id='Heatmap2')

<h3> Show Normals </h3>

In [ ]:
pts = random_mesh.points[lms_indices]
vrt = np.zeros((random_mesh.n_points,3))
vrt[lms_indices] = random_mesh.vertex_normals()[lms_indices] / 5

In [ ]:
random_mesh.view(normals=vrt, 
                 normals_marker_size= 0.5,
                 normals_line_width = 0.01,
                figure_id='Normals')

In [ ]:
random_mesh_landmarks.view(figure_id='Normals', new_figure=False)

In [ ]:
random_mesh.landmarks = random_mesh_landmarks
random_mesh.view( normals=vrt, 
                 normals_marker_size= 0.5,
                 normals_line_width = 0.01)

<h3> Show PointCloud with colours </h3>

In [ ]:
from IPython.utils.tempdir import TemporaryWorkingDirectory
with TemporaryWorkingDirectory() as tmpdir:
    !mkdir -p ./data/PittsburghBridge
    !wget -P ./data/PittsburghBridge https://dl.fbaipublicfiles.com/pytorch3d/data/PittsburghBridge/pointcloud.npz
    pointcloud = np.load('./data/PittsburghBridge/pointcloud.npz')

In [ ]:
points = pointcloud['verts']
colours = pointcloud['rgb']

coloured_pointcloud = PointCloud(points)

In [ ]:
coloured_pointcloud.view(colours=colours)

<h3> Show Surface </h3>

<h3> Show ColouredTriMesh </h3>


In [ ]:
colors = np.random.rand(random_mesh.n_points)

In [ ]:
colors[0:1000]= 0.1
colors[1000:10000]= 0.2
colors[1000:10000]= 0.4
colors[10000:30000]= 0.6
colors[30000:40000]= 0.8
colors[40000:50000]= 0.8
colors[50000:]= 1

In [ ]:
new_mesh = ColouredTriMesh(random_mesh.points, random_mesh.trilist, colours=colors)
new_mesh.view()

In [ ]:
new_mesh.view(normals=vrt, 
                 normals_marker_size= 0.5,
                 normals_line_width = 0.01,
                figure_id='Normals2')

In [ ]:
new_mesh.landmarks = random_mesh_landmarks

<h3> Show Graphs </h3>

In [ ]:
from menpo.shape import PointUndirectedGraph     
import numpy as np
points = np.array([[10, 30, 10], [0, 20, 11], [20, 20, 11], [0, 10, 12], [20, 10, 12], [0, 0, 12]])                                                                                                                    
edges = np.array([[0, 1], [1, 0], [0, 2], [2, 0], [1, 2], [2, 1], 
                  [1, 3], [3, 1], [2, 4], [4, 2], [3, 4], [4, 3],[3, 5], [5, 3]])  
colors = [
    0xff,
    0xffff,
    0xff00ff,
    0x00ffff,
    0xffff00,]

graph = PointUndirectedGraph.init_from_edges(points, edges)   

In [ ]:
graph.view()

In [ ]:
graph.view(line_colour=colors, render_numbering=True)

<h3> View a Morphable Model </h3>

In [ ]:
model.view(figure_id='Model', n_parameters=10, 
           landmarks_indices=lms_indices, alpha=0.4)

In [ ]:
random_mesh.view()
lms.view(new_figure=False)
mesh.view(new_figure=False)

<h3> Fail cases (supposed you have already executed all the above cells)</h3> 

In [ ]:
# It should fail if the previous cells have been executed, as we have already had a figure with id 
# James and we cannot create a new one with the same figure_id
mesh.view(figure_id='James', new_figure=True)

In [ ]:
# It should fail if the previous cells have been executed, as we have already had a figure with id 
# Model and we cannot create a new one with the same figure_id
model.view(inline=True, figure_id='Model')

<h3> Additional functions </h3>

In [ ]:
from menpo3d.visualize import list_figures, dict_figures, 

In [ ]:
tmp = mesh.view(figure_id='Figure Test')

In [ ]:
tmp.remove_widget()

In [ ]:
dict_figures()

<h3> Testing </h3>

In [ ]:
from ipywidgets import Widget
from menpo3d.visualize.viewk3dwidgets import K3dwidgetsRenderer

In [ ]:
for x in Widget.widgets.values():
    print(type(x),x.model_id)
#    if isinstance(x, K3dwidgetsRenderer):
    if hasattr(x,'figure_id'):
        print(type(x),x.model_id, x.figure_id)

In [ ]:
list_figures()